# Misclassified articles

In this notebook, we explore the articles that were misclassified by our model. 

In [1]:
import pickle
import pandas as pd
import numpy as np
import random

In [2]:
path='/home/maggie/News_classifier/4. Features Creation/' 

# X_test    
with open(path + 'X_test.pickle', 'rb') as file:
    X_test=pickle.load(file)

# features_test
with open(path + 'features_test.pickle', 'rb') as file:
    features_test=pickle.load(file)    
    
# y_test
with open(path + 'y_test.pickle', 'rb') as file:
    y_test=pickle.load(file)
        
# best model
with open('best_lsvc.pickle', 'rb') as file:
    model=pickle.load(file)
    
    

In [3]:
print(X_test.shape)
print(features_test.shape)
print(y_test.shape)

(7170, 2)
(7170, 8257)
(7170,)


In [4]:
#Reverse dictionary - to pass from label codes to labels
reversed_label_codes={}
categories=['Cultura y Espectáculos', 'Deportes', 'Economía','Internacional','Política','Sociedad/Policiales']
for num in range(len(categories)):
    reversed_label_codes[num]=categories[num]

We compute the predictions for our test set and the predicted probabilities.

In [5]:
y_pred=model.predict(features_test)
y_pred_proba=model.predict_proba(features_test)

We create a test dataframe with all the computations.

In [6]:
df_test= X_test
df_test['Label_code']=y_test
df_test['Label'] = df_test['Label_code']
df_test= df_test.replace({'Label':reversed_label_codes})
df_test['Prediction']=y_pred
df_test['Prediction_proba']=y_pred_proba.tolist()
df_test['Label_Prediction']=df_test['Prediction']
df_test= df_test.replace({'Label_Prediction':reversed_label_codes})
df_test=df_test[['Texto_clean','Label','Label_Prediction','Prediction_proba']]

In [7]:
df_test.head()

,Texto_clean,Label,Label_Prediction,Prediction_proba
23690,una bisagra histórica que contrapone el princi...,Política,Internacional,"[0.007658383169950256, 0.002866972456233079, 0..."
24353,rusia acusa a ucrania de torturar a soldados r...,Internacional,Internacional,"[0.0010477795422834904, 0.0019636575858595017,..."
2773,iguales pero distintos la vuelta al trabajo pr...,Sociedad/Policiales,Cultura y Espectáculos,"[0.3481506287369228, 0.0208656571690086, 0.230..."
23950,acuerdo con el fmi cuáles son los pasos a segu...,Economía,Economía,"[0.001675986416664208, 0.00020372804425518006,..."
16107,estados unidos incautó fondos afganos para ayu...,Internacional,Internacional,"[0.005088242368044489, 0.0027825022596589624, ..."


In [8]:
df_test.shape


(7170, 4)

We get the misclassified articles:

In [42]:
df_test_misclassified = df_test[df_test['Label'] != df_test['Label_Prediction']]
df_test_misclassified.head()

,Texto_clean,Label,Label_Prediction,Prediction_proba
23690,una bisagra histórica que contrapone el princi...,Política,Internacional,"[0.007658383169950256, 0.002866972456233079, 0..."
2773,iguales pero distintos la vuelta al trabajo pr...,Sociedad/Policiales,Cultura y Espectáculos,"[0.3481506287369228, 0.0208656571690086, 0.230..."
21106,un fallo judicial frenó la entrega de tierras ...,Sociedad/Policiales,Política,"[0.004894425609192773, 0.0012036417342927547, ..."
18697,el pide al fmi acelerar el nuevo fondo para pa...,Internacional,Política,"[0.0004068938016233832, 0.00027464780778460254..."
16917,gobierno estima que llegará al congreso con el...,Política,Economía,"[0.0002895247966268422, 0.0008975463820244922,..."


Number of misclassified news:

In [46]:
print(df_test_misclassified.shape[0])

1009


In [48]:
df_test_misclassified.groupby(['Label', 'Label_Prediction']).size()

Label                   Label_Prediction      
Cultura y Espectáculos  Deportes                   16
                        Economía                    7
                        Internacional              28
                        Política                    5
                        Sociedad/Policiales        24
Deportes                Cultura y Espectáculos      8
                        Economía                    8
                        Internacional              20
                        Política                    9
                        Sociedad/Policiales        19
Economía                Cultura y Espectáculos      6
                        Deportes                    5
                        Internacional              46
                        Política                   94
                        Sociedad/Policiales        34
Internacional           Cultura y Espectáculos     15
                        Deportes                    7
                        Economía   

We can explore some articles to see what happen. We made a function to print the info.

In [66]:
def show_misclassified_article(row):
    print('Actual Category: %s' %(row['Label']))
    print('Predicted Category: %s' %(row['Label_Prediction'])+'\n')
    print('Probabilities:')
    #we sort the indices by probability, 
    sorted_indices=sorted(range(len(row['Prediction_proba'])), key=lambda k: row['Prediction_proba'][k],reverse=True)
    sorted_categories=[reversed_label_codes[num] for num in sorted_indices]
    #we sort the probabilities, and we round each number up to 2 decimals.
    sorted_probabilities=sorted(row['Prediction_proba'],reverse=True)
    rounded_probabilities=[round(num,2) for num in sorted_probabilities]
    for i in range(len(sorted_indices)):
        print(sorted_categories[i]+':'+str(rounded_probabilities[i]))
    print('\n')
    print('Clean Text: ')
    print('%s' %(row['Texto_clean']))
    print('-'*116)

We choose some random articles to examine. 

In [73]:
list_random=random.sample(range(0, df_test_misclassified.shape[0]-1), 5)

In [74]:
for i in list_random:
    show_misclassified_article(df_test_misclassified.iloc[i])


Actual Category: Internacional
Predicted Category: Economía

Probabilities:
Economía:0.64
Internacional:0.31
Política:0.03
Deportes:0.01
Sociedad/Policiales:0.01
Cultura y Espectáculos:0.0


Clean Text: 
indígenas de ecuador anuncian protestas por tiempo indefinido desde enero la poderosa conaie tomó la decisión tras la falta de respuesta del gobierno de lasso a su pedido de revertir el alza de los precios de los combustibles
--------------------------------------------------------------------------------------------------------------------
Actual Category: Economía
Predicted Category: Política

Probabilities:
Política:0.51
Economía:0.38
Internacional:0.08
Sociedad/Policiales:0.02
Cultura y Espectáculos:0.01
Deportes:0.0


Clean Text: 
el gobierno estudia crear una empresa estatal de alimentos la portavoz presidencial gabriela cerruti explicó que brindará incentivos a pequeños productores de verduras y hortalizas
-------------------------------------------------------------------------

In this cases, we can see that the category is not always clear. Sometimes, the articles talk about themes of two or more categories. And sometimes, the category is mislabeled, for example the third new is clearly about politics, but the actual category (the one from the newspaper) is economy.

These cases will always be present, so we don't pretend that our model has 100% accuracy.